In [32]:
%run utils.py
import os.path as osp
import os
import pandas as pd

In [30]:
df = pd.read_csv(osp.join(DATA_DIR, 'pubmed_abstracts.csv'))
df = df[df['abstract'].notnull() & df['title'].notnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46580 entries, 0 to 52173
Data columns (total 5 columns):
abstract    46580 non-null object
date        17852 non-null object
id          46580 non-null int64
terms       46580 non-null object
title       46580 non-null object
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [40]:
tmp_input_file = '/tmp/d3ner_pubmed_input.txt'
tmp_output_file = osp.join(DATA_DIR, 'd3ner_pubmed_output.txt')
if osp.exists(tmp_input_file):
    os.remove(tmp_input_file)
if osp.exists(tmp_output_file):
    os.remove(tmp_output_file)
    
def prep(text):
    text = re.sub('\\<(sup|sub)\\>([a-zA-Z+-]*)\\</(sup|sub)>', '\\2 ', text)
    text = re.sub('[tT]\\-([cC]ell[s]*|[lL]ymphocyte[s]*)', 'T \\1', text)
    return text.replace('\n', ' ').replace('\r', '')

with open(tmp_input_file, 'w') as fd:
    for i, r in df.iterrows():
        fd.write('{}|t|{}\n'.format(r['id'], prep(r['title'])))
        fd.write('{}|a|{}\n\n'.format(r['id'], prep(r['abstract'])))

In [41]:
!cd /Users/eczech/repos/misc/python/D3NER; /Users/eczech/anaconda3/envs/d3ner2/bin/python main.pyc d3ner_fsu fsu_prge $tmp_input_file $tmp_output_file

/Users/eczech/anaconda3/envs/d3ner2/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.
2019-01-30 15:07:37.534926: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
Load NER model finished
[I] Load SpaCy started.
[I] Load SpaCy finished in 9.706 s.


In [43]:
reader = BioCreativeReader(tmp_output_file)

In [45]:
data = reader.read_entity()
len(data.keys())

41071

In [47]:
df = pd.DataFrame([v for e in data.values() for v in e], columns=['id', 'start', 'end', 'value', 'class'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454020 entries, 0 to 454019
Data columns (total 5 columns):
id       454020 non-null object
start    454020 non-null int64
end      454020 non-null int64
value    454020 non-null object
class    454020 non-null object
dtypes: int64(2), object(3)
memory usage: 17.3+ MB


In [48]:
df.head()

,id,start,end,value,class
0,30554495,180,183,CD4,protein
1,30456753,103,110,FcGRIIb,protein
2,30456753,449,456,FcGRIIb,protein
3,30456753,599,607,FcGRIIb-,protein
4,30456753,873,882,cytokines,protein


In [49]:
path = osp.join(DATA_DIR, 'pubmed_abstract_proteins.csv')
df.to_csv(path, index=False)
path

'/Users/eczech/tmp/nlp/data/pubmed_abstract_proteins.csv'